In [19]:
import requests
import json
import pandas as pd
import os
import time

def generate_query(first, skip):
  query = """
    query {
      ranking(first: %s, skip: %s) {
        items {
          id
          name
          twitter
          proposalsCount
          votesCount
          followersCount
          website
        }
      }
    }
  """ % (first, skip)
  return query


out_dir = 'cached_snapshot_data_2023_07_20'
backoff_seconds = 60 / 100 # 100 requests per minute

def get_all_deployments():
    URL = "https://hub.snapshot.org/graphql"

    if not os.path.exists(out_dir):
      os.mkdir(out_dir)

    has_next_page = True
    skip = 0
    while has_next_page:
        # if already cached, skip
        if os.path.exists(f'{out_dir}/snapshot_organizations_{skip}.json'):
            skip += 20
            # print('skipping', skip)
            continue
        query = generate_query(20, skip)
        print('.', skip)
        r = requests.post(URL, json={'query': query})
        time.sleep(backoff_seconds)
        data = json.loads(r.text)

        print(data)
        
        organizations = data['data']['ranking']['items']

        if not organizations:
            print('No organizations found')
            if data.get('errors'):
                print(data['errors'])
                if 'The `skip` argument must not be greater than 15000' in data['errors'][0]['message']:
                    break

        # save to out_dir
        with open(f'{out_dir}/snapshot_organizations_{skip}.json', 'w') as f:
            json.dump(organizations, f)
        
        print('  ', len(organizations))
        if len(organizations) < 20:
            has_next_page = False
        skip += 20


df = get_all_deployments()

. 27480
{'data': {'ranking': {'items': [{'id': 'zenben.eth', 'name': 'ZK Dao', 'twitter': None, 'proposalsCount': 0, 'votesCount': 0, 'followersCount': 0, 'website': None}, {'id': 'zerrox.eth', 'name': 'Zerox', 'twitter': None, 'proposalsCount': 0, 'votesCount': 0, 'followersCount': 0, 'website': None}, {'id': 'zhaocs.eth', 'name': 'zhaocs', 'twitter': None, 'proposalsCount': 0, 'votesCount': 0, 'followersCount': 0, 'website': None}, {'id': 'zhaoyun.eth', 'name': 'zhaoyun.eth', 'twitter': None, 'proposalsCount': 0, 'votesCount': 0, 'followersCount': 0, 'website': None}, {'id': 'zheko87.eth', 'name': 'test', 'twitter': None, 'proposalsCount': 0, 'votesCount': 0, 'followersCount': 0, 'website': None}, {'id': 'zhena.eth', 'name': 'zhena.eth', 'twitter': None, 'proposalsCount': 0, 'votesCount': 0, 'followersCount': 0, 'website': None}, {'id': 'zhenka.eth', 'name': 'Evgeny', 'twitter': None, 'proposalsCount': 0, 'votesCount': 0, 'followersCount': 0, 'website': None}, {'id': 'zhouzhenwen.eth

In [20]:
# load all cached data into a df
files = os.listdir(out_dir)
files = [f for f in files if 'snapshot_organizations' in f]
dfs = []
for f in files:
    with open(f'{out_dir}/{f}', 'r') as f:
        data = json.load(f)
    dfs.append(pd.DataFrame(data))
# reset index
df = pd.concat(dfs)
df

,id,name,twitter,proposalsCount,votesCount,followersCount,website
0,ethfund.eth,ETHFUND,None,1,1,4,None
1,fadedfate.eth,Magic Girls,liuhaha5308848,3,1,3,None
2,fangnaoke.eth,study group,None,1,1,4,https://app.syndicate.io/collectives/0x5f51845...
3,fasola.eth,FASOLA.ETH,None,1,7,1,None
4,fdsfsfss.eth,alex,None,3,3,2,None
...,...,...,...,...,...,...,...
15,sbf666.eth,SBFDAO,None,1,6,15,None
16,stakerocketpool.eth,StakeRocketPool,StakeRocketPool,11,16,5,https://www.stakerocketpool.xyz
17,temboguardians.eth,Tembo Guardians,TemboGuardians,2,31,2,None
18,visionaryboy.eth,Fantom Boy,FantomBOYbyVB,3,22,6,None


In [21]:
# how many have less than 10 proposals?
(df['proposalsCount'] < 10).value_counts()
# preview of events to come...

True     25254
False     2245
Name: proposalsCount, dtype: int64

In [22]:
# combine twitter and website into an 'additional' column, ignore NaNs
df['additional'] = df[['twitter', 'website']].apply(lambda x: ', '.join(x.dropna()), axis=1)
# drop twitter and website columns
df.drop(['twitter', 'website'], axis=1, inplace=True)
df

,id,name,proposalsCount,votesCount,followersCount,additional
0,ethfund.eth,ETHFUND,1,1,4,
1,fadedfate.eth,Magic Girls,3,1,3,liuhaha5308848
2,fangnaoke.eth,study group,1,1,4,https://app.syndicate.io/collectives/0x5f51845...
3,fasola.eth,FASOLA.ETH,1,7,1,
4,fdsfsfss.eth,alex,3,3,2,
...,...,...,...,...,...,...
15,sbf666.eth,SBFDAO,1,6,15,
16,stakerocketpool.eth,StakeRocketPool,11,16,5,"StakeRocketPool, https://www.stakerocketpool.xyz"
17,temboguardians.eth,Tembo Guardians,2,31,2,TemboGuardians
18,visionaryboy.eth,Fantom Boy,3,22,6,FantomBOYbyVB


In [23]:
# add platform = 'snapshot'
df['platform'] = 'snapshot'
# rename id to platform_id
df.rename(columns={'id': 'platform_id'}, inplace=True)
df

,platform_id,name,proposalsCount,votesCount,followersCount,additional,platform
0,ethfund.eth,ETHFUND,1,1,4,,snapshot
1,fadedfate.eth,Magic Girls,3,1,3,liuhaha5308848,snapshot
2,fangnaoke.eth,study group,1,1,4,https://app.syndicate.io/collectives/0x5f51845...,snapshot
3,fasola.eth,FASOLA.ETH,1,7,1,,snapshot
4,fdsfsfss.eth,alex,3,3,2,,snapshot
...,...,...,...,...,...,...,...
15,sbf666.eth,SBFDAO,1,6,15,,snapshot
16,stakerocketpool.eth,StakeRocketPool,11,16,5,"StakeRocketPool, https://www.stakerocketpool.xyz",snapshot
17,temboguardians.eth,Tembo Guardians,2,31,2,TemboGuardians,snapshot
18,visionaryboy.eth,Fantom Boy,3,22,6,FantomBOYbyVB,snapshot


In [24]:
# add website column https://snapshot.org/#/ + platform_id
df['website'] = 'https://snapshot.org/#/' + df['platform_id']
df

,platform_id,name,proposalsCount,votesCount,followersCount,additional,platform,website
0,ethfund.eth,ETHFUND,1,1,4,,snapshot,https://snapshot.org/#/ethfund.eth
1,fadedfate.eth,Magic Girls,3,1,3,liuhaha5308848,snapshot,https://snapshot.org/#/fadedfate.eth
2,fangnaoke.eth,study group,1,1,4,https://app.syndicate.io/collectives/0x5f51845...,snapshot,https://snapshot.org/#/fangnaoke.eth
3,fasola.eth,FASOLA.ETH,1,7,1,,snapshot,https://snapshot.org/#/fasola.eth
4,fdsfsfss.eth,alex,3,3,2,,snapshot,https://snapshot.org/#/fdsfsfss.eth
...,...,...,...,...,...,...,...,...
15,sbf666.eth,SBFDAO,1,6,15,,snapshot,https://snapshot.org/#/sbf666.eth
16,stakerocketpool.eth,StakeRocketPool,11,16,5,"StakeRocketPool, https://www.stakerocketpool.xyz",snapshot,https://snapshot.org/#/stakerocketpool.eth
17,temboguardians.eth,Tembo Guardians,2,31,2,TemboGuardians,snapshot,https://snapshot.org/#/temboguardians.eth
18,visionaryboy.eth,Fantom Boy,3,22,6,FantomBOYbyVB,snapshot,https://snapshot.org/#/visionaryboy.eth


In [25]:
# rename votesCount to votes_count and proposalsCount to proposals_count
df.rename(columns={'votesCount': 'votes_count', 'proposalsCount': 'proposals_count'}, inplace=True)
# reorder columns
df = df[['platform', 'platform_id', 'name', 'additional', 'website', 'votes_count', 'proposals_count']]
df

,platform,platform_id,name,additional,website,votes_count,proposals_count
0,snapshot,ethfund.eth,ETHFUND,,https://snapshot.org/#/ethfund.eth,1,1
1,snapshot,fadedfate.eth,Magic Girls,liuhaha5308848,https://snapshot.org/#/fadedfate.eth,1,3
2,snapshot,fangnaoke.eth,study group,https://app.syndicate.io/collectives/0x5f51845...,https://snapshot.org/#/fangnaoke.eth,1,1
3,snapshot,fasola.eth,FASOLA.ETH,,https://snapshot.org/#/fasola.eth,7,1
4,snapshot,fdsfsfss.eth,alex,,https://snapshot.org/#/fdsfsfss.eth,3,3
...,...,...,...,...,...,...,...
15,snapshot,sbf666.eth,SBFDAO,,https://snapshot.org/#/sbf666.eth,6,1
16,snapshot,stakerocketpool.eth,StakeRocketPool,"StakeRocketPool, https://www.stakerocketpool.xyz",https://snapshot.org/#/stakerocketpool.eth,16,11
17,snapshot,temboguardians.eth,Tembo Guardians,TemboGuardians,https://snapshot.org/#/temboguardians.eth,31,2
18,snapshot,visionaryboy.eth,Fantom Boy,FantomBOYbyVB,https://snapshot.org/#/visionaryboy.eth,22,3


In [26]:
# save to csv
df.to_csv('snapshot_deployments.csv', index=False)